In [ ]:
#Import the libraries
import math
import pandas_datareader as web
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
import math
import matplotlib.pyplot as plt
import keras
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import *
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping

In [ ]:
#Creating yesterday date stamp:

from datetime import datetime, timedelta

yesterday = datetime.now() - timedelta(days=1)
yesterday_str =yesterday.strftime('%Y-%m-%d')

In [ ]:
#Creating 6 days in the past date stamp:

from datetime import datetime, timedelta

past6days = datetime.now() - timedelta(days=6)
past6days_str =past6days.strftime('%Y-%m-%d')

In [ ]:
past6days_str

In [ ]:
#Creating today date stamp:

from datetime import date
today = date.today()
print(today)

In [ ]:
sl = ['ADBE', 'FVRR', 'GME', 'IDXX','TMO']

In [ ]:
sl

In [ ]:
#for ids in range(len(sl)):
 #   m =sl[ids]
    
    

In [ ]:

#p=[]
#for ids in sl:
#    print(ids)
#    p.append(ids)
    
#p

> BUILDING THE MODELs and making prediction:

In [ ]:
todaypred_allstocks =[]
yesterdaypred_allstocks =[]
diff_allstocks=[]
ratio_allstocks = []
apple_quote_pred =[]
Final_value_allstocks=[]

for ids in range(len(sl)):
##BUILD MODEL:
    #Get the stock quote 
    df = web.DataReader(sl[ids], data_source='yahoo', start='2010-01-01', end=yesterday_str) 
    #Create a new dataframe with only the 'Close' column
    data = df.filter(['Close'])
    #Converting the dataframe to a numpy array
    dataset = data.values
    #Get /Compute the number of rows to train the model on
    training_data_len = math.ceil( len(dataset) *.8) 
    #Scale the all of the data to be values between 0 and 1 
    scaler = MinMaxScaler(feature_range=(0, 1)) 
    scaled_data = scaler.fit_transform(dataset)
    #Create the scaled training data set 
    train_data = scaled_data[0:training_data_len  , : ]
    #Split the data into x_train and y_train data sets
    x_train=[]
    y_train = []
    for i in range(60,len(train_data)):
        x_train.append(train_data[i-60:i,0])
        y_train.append(train_data[i,0])
    #Convert x_train and y_train to numpy arrays
    x_train, y_train = np.array(x_train), np.array(y_train)
    #Reshape the data into the shape accepted by the LSTM
    x_train = np.reshape(x_train, (x_train.shape[0],x_train.shape[1],1))
    #Build the LSTM network model
  #  model = Sequential()
  #  model.add(LSTM(units=50, return_sequences=True,input_shape=(x_train.shape[1],1)))
  #  model.add(LSTM(units=50, return_sequences=False))
  #  model.add(Dense(units=25))
  #  model.add(Dense(units=1))
    #Compile the model
  #  model.compile(optimizer='adam', loss='mean_squared_error')
    #Train the model
  #  model.fit(x_train, y_train, batch_size=1, epochs=1)
    
    ##
    model = Sequential()
    #Adding the first LSTM layer and some Dropout regularisation
    model.add(LSTM(units = 50, return_sequences = True, input_shape = (x_train.shape[1], 1)))
    model.add(Dropout(0.2))
    # Adding a second LSTM layer and some Dropout regularisation
    model.add(LSTM(units = 50, return_sequences = True))
    model.add(Dropout(0.2))
    # Adding a third LSTM layer and some Dropout regularisation
    model.add(LSTM(units = 50, return_sequences = True))
    model.add(Dropout(0.2))
    # Adding a fourth LSTM layer and some Dropout regularisation
    model.add(LSTM(units = 50))
    model.add(Dropout(0.2))
    # Adding the output layer
    model.add(Dense(units = 1))

    # Compiling the RNN
    model.compile(optimizer = 'adam', loss = 'mean_squared_error')

# Fitting the RNN to the Training set
    model.fit(x_train, y_train, epochs = 100, batch_size = 32)
    ##TEST
    #Test data set
    test_data = scaled_data[training_data_len - 60: , : ]
    #Create the x_test and y_test data sets
    x_test = []
    y_test =  dataset[training_data_len : , : ] 
    for i in range(60,len(test_data)):
        x_test.append(test_data[i-60:i,0])

    #Convert x_test to a numpy array 
    x_test = np.array(x_test)
    #Reshape the data into the shape accepted by the LSTM
    x_test = np.reshape(x_test, (x_test.shape[0],x_test.shape[1],1))
    #Getting the models predicted price values
    predictions = model.predict(x_test) 
    predictions = scaler.inverse_transform(predictions)#Undo scaling
    

    
    ###BUILD PREDICTIONS 6 days ago:
#Get the quote
    apple_quote = web.DataReader(sl[ids], data_source='yahoo', start='2010-01-01', end=past6days_str)
    #Create a new dataframe
    new_df = apple_quote.filter(['Close'])
    #Get teh last 60 day closing price 
    last_60_days = new_df[-60:].values
    
    #Scale the data to be values between 0 and 1
    scaler = MinMaxScaler(feature_range=(0, 1)) 
    
    #Converting the dataframe to a numpy array
    dataset = new_df.values
    scaled_data = scaler.fit_transform(dataset)
    last_60_days_scaled = scaler.transform(last_60_days)
    
    #Create an empty list
    X_test = []
    
    #Append teh past 60 days
    X_test.append(last_60_days_scaled)
    
    #Convert the X_test data set to a numpy array
    X_test = np.array(X_test)
    
    #Reshape the data
    X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
    #Get the predicted scaled price
    pred_price = model.predict(X_test)
    #undo the scaling 
    pred_price = scaler.inverse_transform(pred_price)
    #print(pred_price)
    yesterdaypred_allstocks.append(pred_price)
    print(yesterdaypred_allstocks[ids])
    
    ###BUILD PREDICTIONS today:
    #Get the quote
    apple_quote = web.DataReader(sl[ids], data_source='yahoo', start='2010-01-01', end=today)
    #fino al giorno prima di predizione che voglio (predizione del quattro)
    #Create a new dataframe
    new_df = apple_quote.filter(['Close'])
    #Get teh last 60 day closing price 
    last_60_days = new_df[-60:].values
    
    #Scale the data to be values between 0 and 1
    scaler = MinMaxScaler(feature_range=(0, 1)) 
    
    #Converting the dataframe to a numpy array
    dataset = new_df.values
    scaled_data = scaler.fit_transform(dataset)
    last_60_days_scaled = scaler.transform(last_60_days)
    
    #Create an empty list
    X_test = []
    
    #Append teh past 60 days
    X_test.append(last_60_days_scaled)
    
    #Convert the X_test data set to a numpy array
    X_test = np.array(X_test)
    
    #Reshape the data
    X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
    #Get the predicted scaled price
    pred_price = model.predict(X_test)
    #undo the scaling 
    pred_price = scaler.inverse_transform(pred_price)
    #print(pred_price)
    todaypred_allstocks.append(pred_price)
    print(todaypred_allstocks[ids])
    diff_stock = todaypred_allstocks[ids]- yesterdaypred_allstocks[ids]
    diff_allstocks.append(diff_stock)
    print(diff_allstocks[ids])
    ratio_change = (diff_stock)/todaypred_allstocks[ids]
    ratio_allstocks.append(ratio_change)
   # print(ratio_allstocks[ids])
    #real_valudiff_allstocksetoday = web.DataReader(sl[ids], data_source='yahoo', start=today, end=today)
    #real_valuetoday = real_valuetoday.filter(['Close'])

   # Final_value= real_valuetoday + ratio_change*real_valuetoday
    #Final_value_allstocks.append(Final_value)
    #print(Final_value_allstocks[ids])
    


In [ ]:
diff_allstocks

['ADBE', 'FVRR', 'GME', 'IDXX','TMO']

In [ ]:
ratio_allstocks

In [ ]:
ratio_change

In [ ]:
real_valuetoday = web.DataReader('AAPL', data_source='yahoo', start=today, end=today)
real_valuetoday

In [ ]:
diff_allstocks/200


In [ ]:
#ha appena fatto predizione del 
#tre 101.51 
#e del quattro 100.54

In [ ]:
todaypred_allstocks

In [ ]:
yesterdaypred_allstocks

In [ ]:
appl 104 107

In [ ]:
##Get the quote
apple_quote2 = web.DataReader('AAPL', data_source='yahoo', start='2020-11-04', end='2020-11-04')
print(apple_quote2['Close'])

In [ ]:
apple= 114.95  119.03

tsl = 421 438

